## Estructura de la Aplicación:

- **Verificación del backup:**
    - Detectar la presencia del archivo DBSIF.FBK.
    - Verificar la fecha y hora de creación/modificación.
    - Obtener el tamaño del archivo.

- **Notificación vía Email**
    - Si el backup es exitoso (el archivo cumple con los criterios), NO se envía la notificación.
    - En caso de fallo, archivo no se genera, tamaño menor al último se envía una notificación vía email.

- **Configuraciones**
- En el archivo `config.ini` se guardan los parámetros que definen el funcionamiento de la aplicación.


**Requisitos:**

- Bibliotecas necesarias:
    - `os` para verificar el archivo.
    - `smtplib` para enviar emails.
    - `schedule` o utilizar una tarea programada en Windows para ejecutar la verificación.

## Configuraciones en cuenta Gmail 
- **Seguridad:** Nunca debes exponer contraseñas directamente en el código, ya que esto pone en riesgo tus credenciales. Usa variables de entorno o servicios seguros para gestionar tus credenciales.

- **Acceso a Gmail:** Gmail puede bloquear el acceso de aplicaciones no seguras. Es probable que necesites configurar Gmail para permitir aplicaciones menos seguras, o mejor aún, crear una contraseña de aplicaciones si tienes la verificación en dos pasos activada.

Para mejorar la seguridad y funcionamiento:

1- **Uso de variables de entorno** para la contraseña: Puedes almacenar tu contraseña en una variable de entorno en lugar de escribirla directamente en el código. Aquí un ejemplo de cómo hacerlo:

Puedes establecer la variable de entorno `EMAIL_PASSWORD` en tu sistema de la siguiente manera:

- En Windows:
`setx EMAIL_PASSWORD "tu_contraseña"`

- En Linux/MacOS:
`export EMAIL_PASSWORD="tu_contraseña"`

2- **Habilitar aplicaciones menos seguras en Gmail**: Si no tienes activada la verificación en dos pasos, puedes permitir que aplicaciones menos seguras accedan a tu cuenta de Gmail:
- Ve a Permitir el acceso de aplicaciones menos seguras y actívalo.

3- **Uso de contraseñas de aplicación**: Si tienes activada la autenticación de dos pasos, lo ideal sería generar una contraseña de aplicación específica para tu script, en lugar de utilizar tu contraseña principal.

## Verificación del archivo DBSIF.FBK:


In [5]:
# Importamos librerías

import os
import time

import smtplib
import ssl
import configparser
import json

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


## Configuraciones con config.ini
- email envío
- contraseña con variables de entorno
- Ruta de la DB
- email destino

Para configurar los parámetros de tu aplicación en un archivo `config.ini`, puedes utilizar el módulo `configparser de Python`, que facilita la lectura y escritura de archivos de configuración.

El archivo `config.ini` sigue una estructura en la que los parámetros se agrupan en secciones, y cada sección contiene claves y valores. A continuación, te muestro un ejemplo básico y cómo utilizarlo en tu código Python:

1. **Ejemplo de archivo config.ini:**

```python
[Database]
host = localhost
port = 5432
user = admin
password = secret

[Logging]
log_level = INFO
log_file = /var/log/myapp.log

[API]
endpoint = https://api.example.com
api_key = your-api-key
timeout = 30
```

- [Database]: Parámetros relacionados con la base de datos.
- [Logging]: Configuraciones de registro/logging.
- [API]: Configuración para acceso a una API.

2. **Cómo leer el archivo config.ini desde Python:**
Primero, necesitas importar el módulo configparser y luego leer el archivo config.ini para acceder a los valores.

```Python

import configparser

# Crear un objeto ConfigParser
config = configparser.ConfigParser()

# Leer el archivo de configuración
config.read('config.ini')

# Acceder a los parámetros
db_host = config['Database']['host']
db_port = config['Database'].getint('port')  # convertimos a entero
db_user = config['Database']['user']
db_password = config['Database']['password']

log_level = config['Logging']['log_level']
log_file = config['Logging']['log_file']

api_endpoint = config['API']['endpoint']
api_key = config['API']['api_key']
timeout = config['API'].getint('timeout')  # convertimos a entero

# Mostrar algunos valores para verificar
print(f"Connecting to database on {db_host}:{db_port} as {db_user}")
print(f"Logging at {log_level} level to {log_file}")
print(f"API endpoint: {api_endpoint}, Timeout: {timeout} seconds") 
```


3. **Ventajas de usar config.ini:**

- *Separación de código y configuración:*
Los parámetros de configuración no están embebidos en el código, lo que facilita su modificación sin tocar la lógica de la aplicación.

- *Facilidad de uso:*
Es simple de leer y modificar tanto para humanos como para programas.
- *Flexibilidad:* Puedes agregar nuevas secciones y parámetros sin cambiar el código de la aplicación.







In [ ]:

# Crear un objeto ConfigParser
config = configparser.ConfigParser()

# Leer el archivo de configuración
config.read('config.ini')

# Acceder a los parámetros

# Cliente
cliente = config['Account']['client']

# Ruta de la DB
backup_path = config['Backup']['path']
# Opción: Asegurar que las barras estén correctamente escapadas
backup_path = backup_path.replace("\\", "/")  # o usa "\\\\" para barras dobles

email_origin = config['Email']['email_origin']
email_destination = config['Email']['email_destination']
email_cc = config['Email']['email_cc'] if config['Email']['email_cc'] else None
email_cco = config['Email']['email_cco'] if config['Email']['email_cco'] else None

# Obtener la variable de entorno para la clave del correo
email_password = os.getenv(config['Email']['env_variable_key'])

# Obtiene el nombre del archivo JSON que se genera para guardar log del backup
json_file_path = config['Json']['path']

# Mostrar algunos valores para verificar
print(f"Cliente: {cliente}")
print(f"Backup path: {backup_path}")
print(f"Email Origin: {email_origin}")
print(f"Email Destination: {email_destination}")
print(f"Email CC: {email_cc}")
print(f"Email CCO: {email_cco}")
print(f"Email Password (from env): {email_password if email_password else 'No environment variable set'}")
print(f"El archivo Json es: {json_file_path}")



In [7]:
def enviar_correo(email_origin, email_password, email_destino, asunto, cuerpo):
    # Crear el mensaje de correo
    msg = MIMEText(cuerpo)
    msg['Subject'] = asunto
    
    msg['From'] = email_origin
    msg['To'] = email_destino

    # Enviar el correo
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(email_origin, email_password)
            server.sendmail(email_origin, [email_destino], msg.as_string())
        print("Correo enviado con éxito.")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

In [ ]:
def verificar_backup(ruta_backup, email_origin, email_password, email_destino):
    # Verificar si el archivo de backup existe
    if os.path.exists(ruta_backup):
        # Obtener el tamaño en GB y la fecha de creación/modificación
        tamano_actual = os.path.getsize(ruta_backup) / (1024 * 1024 * 1024)
        timestamp_actual = os.path.getmtime(ruta_backup)
        fecha_actual = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp_actual))
        codigo_backup = "backup_ok" # inicializador de código

        # Leer el archivo JSON o crear uno nuevo si no existe
        if os.path.exists(json_file_path):
            with open(json_file_path, 'r') as file:
                registros_backup = json.load(file)
        else:
            registros_backup = []

        # Verificar el último registro (si existe)
        if registros_backup:
            ultimo_registro = registros_backup[-1]  # tomo el último backup registrado
            tamano_anterior = ultimo_registro['tamano_gb']
            fecha_anterior = ultimo_registro['fecha']
            codigo_backup = ultimo_registro['codigo_backup']

            # Comparar tamaño y fecha del backup actual con el anterior
            if tamano_actual > tamano_anterior and fecha_actual >= fecha_anterior:
                codigo_backup = "backup_ok"
            else:
                # Hay ERROR
                # Inicializamos el código de error
                codigo_backup = "backup_Error"
                # Detectamos si hay un problema con el tamaño
                if tamano_actual <= tamano_anterior:
                    codigo_backup += "Tamaño <="
    
                # Detectamos si hay un problema con la fecha
                if fecha_actual < fecha_anterior:
                    codigo_backup += "Fecha < ultimo_backup. "

            # Preparar el asunto y cuerpo del correo
            asunto = "ERROR al verificar Backup"
            # Definimos un ancho fijo para cada columna
            col_ancho_1 = 20
            col_ancho_2 = 35
            col_ancho_3 = 35
            # Generamos el cuerpo del mensaje con las columnas alineadas
            cuerpo = (
                f"El backup actual no cumple con los criterios de verificación:\n\n"
                f"{'Descripción'.ljust(col_ancho_1)}{'Valor Actual'.ljust(col_ancho_2)}{'Valor Anterior'.ljust(col_ancho_3)}\n"
                f"{'-' * (col_ancho_1 + col_ancho_2 + col_ancho_3)}\n"
                f"{'Tamaño'.ljust(col_ancho_1)}{f'{tamano_actual:.2f} GB'.ljust(col_ancho_2)}{f'{tamano_anterior:.2f} GB'.ljust(col_ancho_3)}\n"
                f"{'Fecha'.ljust(col_ancho_1)}{f'{fecha_actual}'.ljust(col_ancho_2)}{f'{fecha_anterior}'.ljust(col_ancho_3)}\n"
                f"{'-' * (col_ancho_1 + col_ancho_2 + col_ancho_3)}\n"
                f"Resultado: {codigo_backup}\n"
            )

            # Llamada para enviar el correo de advertencia
            enviar_correo(email_origin, email_password, email_destino, asunto, cuerpo)


        # Actualizar o añadir el registro actual en el archivo JSON
        nuevo_registro = {"tamano_gb": tamano_actual, "fecha": fecha_actual, "codigo_backup":codigo_backup }
        registros_backup.append(nuevo_registro)

        # Guardar el nuevo registro en el archivo JSON
        with open(json_file_path, 'w') as file:
            json.dump(registros_backup, file, indent=4)

        return {"existe": True, "tamano_gb": tamano_actual, "fecha": fecha_actual, "codigo_backup":codigo_backup}
    else:
        return {"existe": False}
    


# Cuerpo principal

In [9]:


verificar_backup(backup_path, email_origin, email_password, email_destination)

Correo enviado con éxito.


{'existe': True,
 'tamano_gb': 0.04587364196777344,
 'fecha': '2024-02-05 10:25:00',
 'codigo_backup': 'backup_ErrorTamaño <='}